# Compare mean and variability in control and stim states

In [1]:
import scanpy as sc
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
from pybedtools import BedTool
import pickle as pkl
%matplotlib inline

In [2]:
import sys
sys.path.append('/home/ssm-user/Github/scrna-parameter-estimation/dist/memento-0.0.5-py3.8.egg')
sys.path.append('/home/ssm-user/Github/misc-seq/miscseq/')
import encode
import memento

In [4]:
data_path = '/data_volume/memento/hbec/'

### Read the processed RNA data

Focus on the club and bc/club cells and type I interferons for now.

Encode the timestamps to integers.

In [5]:
adata = sc.read(data_path + 'HBEC_type_I_filtered_counts_deep.h5ad')

In [6]:
adata = adata[:, ~adata.var.index.str.startswith('MT-')].copy()
adata.obs['cell_type'] = adata.obs['cell_type'].apply(lambda x: x if x != 'basal/club' else 'bc')
adata.obs['cell_type'] = adata.obs['cell_type'].apply(lambda x: x if x != 'ionocyte/tuft' else 'ion-tuft')

/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [7]:
adata.shape

(69958, 36588)

### Setup memento

In [8]:
def assign_q(batch):
    
    if batch == 0:
        return 0.387*0.25
    elif batch == 1:
        return 0.392*0.25
    elif batch == 2:
        return 0.436*0.25
    else:
        return 0.417*0.25

In [9]:
adata.obs['q'] = adata.obs['batch'].apply(assign_q)

In [10]:
memento.setup_memento(adata, q_column='q')

In [11]:
adata.obs.head(3)

NUM.SNPS BEST.GUESS DROPLET.TYPE batch HTO_classification  \
AAACCCAAGGAAGTAG-1      1700     donor0          SNG     0            hash-10   
AAACCCAAGGGACTGT-1       839     donor0          SNG     0             hash-3   
AAACCCACAATCTGCA-1      1130     donor1          SNG     0             hash-1   

                         condition  donor    stim time  n_genes  \
AAACCCAAGGAAGTAG-1  d2513_lambda_9  d2513  lambda    9     4297   
AAACCCAAGGGACTGT-1  d2513_alpha_24  d2513   alpha   24     2171   
AAACCCACAATCTGCA-1   d2614_alpha_3  d2614   alpha    3     3016   

                    n_genes_by_counts  total_counts  total_counts_mt  \
AAACCCAAGGAAGTAG-1               4297       12191.0           1394.0   
AAACCCAAGGGACTGT-1               2171        7270.0            386.0   
AAACCCACAATCTGCA-1               3016        9544.0            516.0   

                    pct_counts_mt leiden cell_type        q  \
AAACCCAAGGAAGTAG-1      11.434665      3  ciliated  0.10425   
AAACCCAAGGGACTGT-1       5.309491      5        bc  0.10425   
AAACCCACAATCTGCA-1       5.406538      0     basal  0.10425   

                    memento_size_factor  
AAACCCAAGGAAGTAG-1             0.955289  
AAACCCAAGGGACTGT-1             0.824407  
AAACCCACAATCTGCA-1             0.988939

### Run memento for each subset, comparing to control

In [ ]:
cts = [['ciliated']]
tps = ['3', '6', '9', '24', '48']
stims = ['alpha', 'beta', 'gamma', 'lambda']

In [ ]:
import os
done_files = os.listdir('/data_volume/ifn_hbec/binary_test_deep/')

In [ ]:
for ct in cts:
    for tp in tps:
        for stim in stims:
            
            fname = '{}_{}_{}_20200313.h5ad'.format('-'.join(ct), stim, tp)
            
            if fname in done_files:
                print('Skipping', fname)
                continue

            print('starting', ct, tp, stim)

            adata_stim = adata.copy()[
                adata.obs.cell_type.isin(ct) & \
                adata.obs.stim.isin(['control', stim]) & \
                adata.obs.time.isin(['0',tp]), :].copy()
            time_converter={0:0, int(tp):1}
            adata_stim.obs['time_step'] = adata_stim.obs['time'].astype(int).apply(lambda x: time_converter[x])

            memento.create_groups(adata_stim, label_columns=['time_step', 'donor'])
            memento.compute_1d_moments(adata_stim, min_perc_group=.9)

            memento.ht_1d_moments(
                adata_stim, 
                formula_like='1 + time_step',
                cov_column='time_step', 
                num_boot=10000, 
                verbose=1,
                num_cpus=13)

            del adata_stim.uns['memento']['mv_regressor']

            adata_stim.write('/data_volume/ifn_hbec/binary_test_deep/{}_{}_{}_20200313.h5ad'.format(
                '-'.join(ct), stim, tp))

### Run memento for each subset, comparing to control

In [13]:
cts = [['ciliated']]
tps = ['3']
stims = ['alpha', 'beta', 'gamma', 'lambda']

In [14]:
import os
done_files = os.listdir('/data_volume/memento/hbec/isg_classes/')

In [29]:
def export_result(df):
    df['de_fdr'] = memento.util._fdrcorrect(df['de_pval'])
    return df[['gene', 'de_coef', 'de_pval', 'de_fdr']].rename(columns={'de_coef':'log_fc', 'de_pval':'pval', 'de_fdr':'fdr'}).query('fdr < 0.01 & log_fc > 0')

In [28]:
for ct in cts:
    for tp in tps:
        for stim in stims:
            
            fname = '{}_{}_{}.csv'.format('-'.join(ct), stim, tp)
            
            if fname in done_files:
                print('Skipping', fname)
                continue

            print('starting', ct, tp, stim)

            adata_stim = adata.copy()[
                adata.obs.cell_type.isin(ct) & \
                adata.obs.stim.isin(['control', stim]) & \
                adata.obs.time.isin(['0',tp]), :].copy()
            time_converter={0:0, int(tp):1}
            adata_stim.obs['time_step'] = adata_stim.obs['time'].astype(int).apply(lambda x: time_converter[x])

            memento.create_groups(adata_stim, label_columns=['time_step', 'donor'])
            memento.compute_1d_moments(adata_stim, min_perc_group=.4)
            
            memento.ht_1d_moments(
                adata_stim, 
                formula_like='1 + time_step',
                cov_column='time_step', 
                num_boot=10000, 
                verbose=1,
                num_cpus=13)

            result = memento.get_1d_ht_result(adata_stim)
            export_result(result).to_csv('/data_volume/memento/hbec/isg_classes/'+fname, index=False)

starting ['ciliated'] 3 alpha


/home/ssm-user/anaconda3/envs/single_cell/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/home/ssm-user/Github/scrna-parameter-estimation/dist/memento-0.0.5-py3.8.egg/memento/estimator.py:105: RuntimeWarning: invalid value encountered in log
[Parallel(n_jobs=13)]: Using backend LokyBackend with 13 concurrent workers.
[Parallel(n_jobs=13)]: Done  24 tasks      | elapsed:    1.0s
[Parallel(n_jobs=13)]: Done 174 tasks      | elapsed:    5.1s
[Parallel(n_jobs=13)]: Done 424 tasks      | elapsed:   13.4s
[Parallel(n_jobs=13)]: Done 774 tasks      | elapsed:   25.3s
[Parallel(n_jobs=13)]: Done 1224 tasks      | elapsed:   39.6s
[Parallel(n_jobs=13)]: Done 1774 tasks      | elapsed:   56.8s
[Parallel(n_jobs=13)]: Done 2424 tasks      | elapsed:  1.3min
[Parallel(n_jobs=13)]: Done 3174 tasks      | elap

TypeError: 'method' object is not subscriptable